# Скрипт генерирующий итоговое решени

In [23]:
import pandas as pd
import joblib
import numpy as np

# Шаг 1: Загрузка данных
test_data = pd.read_csv('test.csv')  # Предполагается, что у вас есть файл test_data.csv с тестовыми данными

# Добавление фичей к каждому объекту

In [24]:
features = pd.read_csv('features.csv')

In [25]:
from scipy.spatial.distance import cdist

In [26]:
n_closest = 3

# Расчет расстояний между точками
distances = cdist(test_data[['lat', 'lon']], features[['lat', 'lon']], metric='euclidean')

# Находим индексы n самых близких строк
closest_indices = np.argsort(distances, axis=1)[:, :n_closest]

# Получаем ближайшие строки и вычисляем медиану признаков
closest_features = features.iloc[closest_indices.flatten()]
grouping_index = np.repeat(np.arange(len(test_data)), n_closest)
closest_features_grouped = closest_features.groupby(grouping_index).median()

# убираем колонки широты и долготы у новых признаков
closest_features_grouped.drop(['lat', 'lon'], axis=1,inplace=True)

# Объединяем признаки с исходными данными
data_with_features = pd.concat([test_data, closest_features_grouped], axis=1)

In [27]:
data_with_features

,id,lat,lon,0,1,2,3,4,5,6,...,353,354,355,356,357,358,359,360,361,362
0,3084,55.782276,49.148234,0.135929,0.184580,0.206535,0.121892,0.105068,0.106734,0.303218,...,0.055745,0.063922,0.099472,0.109843,0.017741,0.007329,0.057079,0.063922,0.069942,0.084518
1,3085,56.841500,35.853059,0.141960,0.144476,0.266451,0.146753,0.132175,0.130770,0.200901,...,0.040617,0.043006,0.069545,0.050458,0.009615,0.008011,0.124602,0.126093,0.240914,0.289976
2,3086,55.356780,86.156697,0.095802,0.113192,0.246053,0.083412,0.094552,0.079853,0.175057,...,0.291881,0.288932,0.621832,0.584057,0.355769,0.137379,0.460685,0.465570,0.647831,0.695935
3,3087,56.755087,60.703962,0.223629,0.214923,0.388710,0.236384,0.201722,0.228324,0.228532,...,0.038988,0.048350,0.005124,0.004916,0.009074,0.004091,0.040346,0.048350,0.003603,0.003782
4,3088,59.907438,30.250297,0.521361,0.395949,0.621932,0.541924,0.543039,0.510069,0.382604,...,0.042506,0.046563,0.079101,0.079255,0.011647,0.008352,0.053011,0.046563,0.076312,0.086558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,4108,51.692337,39.143301,0.124033,0.140029,0.195563,0.115171,0.110365,0.113758,0.235057,...,0.046838,0.051583,0.077445,0.074362,0.002844,0.007329,0.048185,0.051583,0.054454,0.057218
1025,4109,56.309197,43.987060,0.144990,0.167534,0.242255,0.136429,0.118399,0.134710,0.354277,...,0.043089,0.054659,0.153717,0.142414,0.007449,0.054201,0.044441,0.054659,0.108084,0.109580
1026,4110,59.898712,30.484624,0.395762,0.303573,0.416961,0.485573,0.392372,0.379509,0.353598,...,0.045745,0.049677,0.082257,0.088580,0.007855,0.005625,0.063316,0.049677,0.074984,0.117320
1027,4111,54.991157,73.407459,0.107564,0.149363,0.157945,0.078858,0.109236,0.105524,0.386544,...,0.033483,0.053290,0.172277,0.173267,0.019231,0.008863,0.053054,0.058863,0.166381,0.168324


### Оставим только фичи, которые посчитали подходящими:

In [28]:
# Загрузка списка подходящих столбцов из файла
columns_df = pd.read_csv('suitable_columns.csv')

# Получение списка подходящих столбцов
suitable_columns = columns_df['Column_Name'].tolist()

In [29]:
data_with_features

,id,lat,lon,0,1,2,3,4,5,6,...,353,354,355,356,357,358,359,360,361,362
0,3084,55.782276,49.148234,0.135929,0.184580,0.206535,0.121892,0.105068,0.106734,0.303218,...,0.055745,0.063922,0.099472,0.109843,0.017741,0.007329,0.057079,0.063922,0.069942,0.084518
1,3085,56.841500,35.853059,0.141960,0.144476,0.266451,0.146753,0.132175,0.130770,0.200901,...,0.040617,0.043006,0.069545,0.050458,0.009615,0.008011,0.124602,0.126093,0.240914,0.289976
2,3086,55.356780,86.156697,0.095802,0.113192,0.246053,0.083412,0.094552,0.079853,0.175057,...,0.291881,0.288932,0.621832,0.584057,0.355769,0.137379,0.460685,0.465570,0.647831,0.695935
3,3087,56.755087,60.703962,0.223629,0.214923,0.388710,0.236384,0.201722,0.228324,0.228532,...,0.038988,0.048350,0.005124,0.004916,0.009074,0.004091,0.040346,0.048350,0.003603,0.003782
4,3088,59.907438,30.250297,0.521361,0.395949,0.621932,0.541924,0.543039,0.510069,0.382604,...,0.042506,0.046563,0.079101,0.079255,0.011647,0.008352,0.053011,0.046563,0.076312,0.086558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1024,4108,51.692337,39.143301,0.124033,0.140029,0.195563,0.115171,0.110365,0.113758,0.235057,...,0.046838,0.051583,0.077445,0.074362,0.002844,0.007329,0.048185,0.051583,0.054454,0.057218
1025,4109,56.309197,43.987060,0.144990,0.167534,0.242255,0.136429,0.118399,0.134710,0.354277,...,0.043089,0.054659,0.153717,0.142414,0.007449,0.054201,0.044441,0.054659,0.108084,0.109580
1026,4110,59.898712,30.484624,0.395762,0.303573,0.416961,0.485573,0.392372,0.379509,0.353598,...,0.045745,0.049677,0.082257,0.088580,0.007855,0.005625,0.063316,0.049677,0.074984,0.117320
1027,4111,54.991157,73.407459,0.107564,0.149363,0.157945,0.078858,0.109236,0.105524,0.386544,...,0.033483,0.053290,0.172277,0.173267,0.019231,0.008863,0.053054,0.058863,0.166381,0.168324


In [30]:
data_with_features = data_with_features[suitable_columns]

# Получение предсказания и итогового файла

In [31]:
# Шаг 2: Загрузка обученной модели
model = joblib.load('trained_model.pkl')  # Предполагается, что у вас есть файл trained_model.pkl с обученной моделью

# Шаг 3: Применение модели к данным и получение предсказаний
predictions = model.predict(data_with_features)  # Предполагается, что ваша модель умеет делать предсказания

In [32]:
predictions

array([0.08100731, 0.08346323, 0.11494929, ..., 0.11226609, 0.21096075,
       0.10364364])

In [33]:
result = pd.DataFrame()

In [34]:
result['id'] = test_data['id']
result['score'] = predictions

In [35]:
result.to_csv('submission.csv', index=False)